This script creates geojson filles.

Data was originally downloaded from the
Humanitarian Data Exchange / UN Office for 
the Coordination of Humanitarian Affairs.

They were processed to be in the format
needed for the application to run.

The file dids not originally include the
'Zona en Reclamación'.

https://data.humdata.org/dataset/cod-ab-ven0



In [1]:
import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
import re

#### Helpers for smart rounding

In [11]:
simpledec = re.compile(r"\d*\.\d+")
def mround(match):
    return "{:.6f}".format(float(match.group()))

#### Geographic file processing

Converts province data to GeoJSON.

In [12]:
provincias = gpd.read_file("../input/provincias.zip")
provincias = provincias[['ADM1_ES', 'ADM1_PCODE', 'geometry']]
provincias = provincias.rename(columns={'ADM1_ES': 'name', 'ADM1_PCODE': 'code'})
provincias.loc[24, 'code'] = 'VE99' # Code for Federal Dependencies was nan
provincias['geometry'] = provincias.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))
provincias.to_file("../output/level-1.geojson", driver="GeoJSON")

The second level of the data visualization will include the 'municipios' division for all the states plus the 'paroquias' for Vargas and Distrito Capital. We will combine them into a single file

In [13]:
municipios = gpd.read_file("../input/municipios.zip")

In [14]:
paroquias = gpd.read_file("../input/paroquias.zip")

In [15]:
# All municipios but Libertador (Distrito Capital) and Vargas (La Guaira)
municipios = municipios[~municipios.ADM1_PCODE.isin(['VE01', 'VE24'])]
municipios = municipios[['ADM1_PCODE', 'ADM1_ES', 'ADM2_PCODE', 'ADM2_ES', 'geometry']]
municipios = municipios.rename(columns={
                                        'ADM1_ES': 'parent_name', 
                                        'ADM1_PCODE': 'parent_code',
                                        'ADM2_ES': 'name', 
                                        'ADM2_PCODE': 'code'})

In [16]:
paroquias = paroquias[paroquias.ADM1_PCODE.isin(['VE01', 'VE24'])] # Paroquias in Libertador and Vargas
paroquias = paroquias[['ADM1_PCODE', 'ADM1_ES', 'ADM3_PCODE', 'ADM3_ES', 'geometry']]
paroquias = paroquias.rename(columns={
                                      'ADM1_ES': 'parent_name', 
                                      'ADM1_PCODE': 'parent_code',
                                      'ADM3_ES': 'name', 
                                      'ADM3_PCODE': 'code'
})


In [17]:
level_2 = pd.concat([municipios, paroquias]).sort_values(by='parent_name')

In [18]:
level_2['geometry'] = level_2.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))


In [19]:
level_2.to_file("../output/level-2.geojson", driver="GeoJSON")